In [13]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
#Cargamos el Dataset
file_path = "dataset/dataset.csv"
df = pd.read_csv(file_path)


In [27]:
#Listamos columnas para X y con drop separamos como un filter
X_columns = [
    'latitude_pos_tx', 'longitude_pos_tx', 'elevation_pos_tx', 'fc', 'elevation', 
    'azimuth', 'year', 'mmdd', 'UTI', 'hour', 'delay', 'terrestrial_range', 
    'slant_range', 'final_latitude', 'final_longitude', 'final_elevation'
]
X = df[X_columns]
Y = df.drop(columns=X_columns)

##### Uso drop()
***drop()*** es una función incorporada en Pandas que te permite eliminar una o más **filas o columnas** de un Marco de Datos. Devuelve un nuevo Marco de Datos con las filas o columnas especificadas eliminadas y no modifica el Marco de Datos original en su lugar, a menos que establezcas el parámetro inplace en True .

In [28]:
# Consideramos que las columnas "final_latitude", "final_longitude", "final_elevation"
# No corresponden a parametros de entradas X por lo que lo quitamos.
X = X.drop(columns=['final_latitude','final_longitude','final_elevation'])
# print(X.head())
# print(Y.head())


In [29]:
# Ahora vamos a escalar las columnas de elevaciones, que se encuentran en metros se los pasará a Km
# y luego se les quitará decimas
# print(Y.head())
#Nota: En Y tenemos 3 tipos(latitudes, longitudes, elevations)
#Generamos los nombres de las columnas. 
lat_columns = [f'lat_{i}' for i in range(1,101)]
long_columns = [f'long_{i}' for i in range(1,101)]
elev_columns = [f'elev_{i}' for i in range(1,101)]



In [30]:
#Teniendo los nombres de las columnas las filtro
latitudes = Y[lat_columns]
longitudes = Y[long_columns]
elevations = Y[elev_columns]
max_value = Y[elev_columns].max()
print("maximo valor por columnas\n",max_value)
print("Maximo valor del conjunto de elevaciones:",max_value.max())
# k=1000 # valor a dividir para pasar a Km las elevaciones.
# elevations = (elevations/k).round(5)
# print(type(elevations))
# print(elevations.head())


maximo valor por columnas
 elev_1         967.773438
elev_2       11580.748895
elev_3       23228.155579
elev_4       34941.924124
elev_5       46721.758602
                ...      
elev_96     301178.222656
elev_97     302036.621094
elev_98     302895.019531
elev_99     303753.417969
elev_100    304611.816406
Length: 100, dtype: float64
Maximo valor del conjunto de elevaciones: 304611.8164061997


In [31]:
#df.to_excel("NuevoExcelCompleto_para_ver.xlsx", index = False)

In [32]:
# Ahora quito las elevaciones en m y concateno con las elevaciones en km y redondeado
Y = Y.drop(columns = elev_columns)
Y = pd.concat([Y,elevations],axis = 1)

In [33]:
#Contamos la cantidad de filas que tienen valores distintos de ceros en la columna elev_1
count_non_zero =(Y["elev_1"]!=0).sum()
print("La cantidad de filas con elev_1 != 0 es",count_non_zero)
count_non_zero_elev_100 = (Y["elev_100"]!=0).sum()
print("Cantidad de filas que no finalizan en cero: ", count_non_zero_elev_100)
elev_101 = np.array([])
for i in Y["elev_100"]:
  if i != 0:
    elev_101 = np.append(elev_101,1)
  else:
    elev_101 = np.append(elev_101,0)
print(elev_101,"cantidad: ", len(elev_101))
# Creamos un DataFrame de 0 y 1 para agregar un elev_101 PERO NO ES NECESARIO.
elev_101_df = pd.DataFrame()
elev_101_df["elev_101"] = elev_101
print(elev_101_df)

La cantidad de filas con elev_1 != 0 es 280
Cantidad de filas que no finalizan en cero:  1519
[0. 0. 0. ... 1. 1. 1.] cantidad:  5130
      elev_101
0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
...        ...
5125       1.0
5126       1.0
5127       1.0
5128       1.0
5129       1.0

[5130 rows x 1 columns]


##### División de los datos en conjuntos de entrenamiento y prueba
  `train_test_split` es una función que forma parte de la biblioteca de scikit-learn en el módulo `sklearn.model_selection`.
  Se lo usa comunmente para dividir un conjunto de datos en dos subconjuntos (entrenamiento y prueba)
#### Sus parámetros
* **Datos**(obligatorio): Conjunto de datos que se desea dividir. 
  ```python
    train_test_split(X,Y)
* **test_size**(opcional): Define el porcentaje o la cantidad de datos que se reserva para prueba. Si es 0.2 indica un 20% reservado para prueba.
  ```python
    train_test_split(X,Y, test_size=0.2)
* **train_size**(opcional): Especifica el tamaño del conjunto de Entrenamiento.
* **random_state**(opcional): Controla la aleatoridad de la división. Si se usa un valor fijo (`random_state=42`), la división siempre es la misma cada vez que se ejecute el código.
  ```python
    train_test_split(X, Y, test_size=0.2, random_state=42)
* **shuffle** (opcional): Especificsa si los datos deben ser barajados antes de ser divididos. Por defecto está en `True`. Es decir, los datos se mezclarán aleatoriamente antes de ser divididos.
  


In [34]:
Y["elev_1"] = Y["elev_1"].apply(lambda x: 0.0 if x != 0 else x)
print(Y["elev_1"].max().max())

0.0


In [38]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

print("Tipo de dato:",type(x_train),"\n", x_train.head())

Tipo de dato: <class 'pandas.core.frame.DataFrame'> 
       latitude_pos_tx  longitude_pos_tx  elevation_pos_tx          fc  \
1235           -42.28             -63.4                 0  10000000.0   
4047           -42.28             -63.4                 0  20000000.0   
949            -42.28             -63.4                 0  10000000.0   
4375           -42.28             -63.4                 0  23000000.0   
19             -42.28             -63.4                 0  10000000.0   

      elevation  azimuth    year  mmdd  UTI  hour     delay  \
1235          5       98  2010.0   725    0    20  0.009413   
4047         28       89  2010.0  1215    0    12  0.002455   
949           5       98  2010.0   608    0     4  0.009073   
4375         34       87  2010.0  1215    0    12  0.001921   
19            5       98  2010.0   104    0     4  0.009910   

      terrestrial_range   slant_range  
1235       2.719923e+06  2.822081e+06  
4047       6.188348e+05  7.360000e+05  
949     

In [36]:
count_non_zero_elev_100 = (y_train["elev_100"]!=0).sum()
print("Cantidad de filas que no finalizan en cero de y_train: ", count_non_zero_elev_100)


Cantidad de filas que no finalizan en cero de y_train:  1213


In [39]:
k = y_train[elev_columns].max().max()
print(k)
#normalizo al numero máximo de todas las columnas de elevaciones de y_train
y_train[elev_columns] = y_train[elev_columns]/k
print(y_train[elev_columns].max())

# Normalizo al maximo del train en el test
y_test[elev_columns] = y_test[elev_columns]/k
print("máximos en el test:",y_test[elev_columns].max().max())

304611.8164061997
elev_1      0.000000
elev_2      0.038018
elev_3      0.076255
elev_4      0.114710
elev_5      0.153381
              ...   
elev_96     0.988728
elev_97     0.991546
elev_98     0.994364
elev_99     0.997182
elev_100    1.000000
Length: 100, dtype: float64
máximos en el test: 0.9997916149046734
